In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import arabicABC as abc
from fuzzywuzzy import fuzz

Deleted character `"` at 5735 and 24523 in column `punc`. This caused issue in reading.
Deleted column `punc`. Deleted all numbers. Deleted all empty rows.
Normalized using `afifi.letters.apply(abc.normalize)`

In [33]:
# afifi = pd.read_csv('AfifiOCR.tsv', sep='\t')

In [2]:
afifi = pd.read_csv('AfifiCleaned.csv')

In [3]:
afifi

,page,stripe,column,line,left,top,right,bottom,confidence,letters
0,47,0,0,1,1785,99,2375,441,56,بنزٕليلالع
1,47,0,0,1,1211,99,1784,441,64,يلرعا
2,47,0,0,2,2841,736,3063,914,94,الحمد
3,47,0,0,2,2763,736,2842,914,99,له
4,47,0,0,2,2569,736,2762,914,94,منزل
...,...,...,...,...,...,...,...,...,...,...
40268,226,0,0,20,1855,3731,1986,3898,99,الله
40269,226,0,0,20,1593,3731,1855,3898,89,عاقبتها
40270,226,0,0,20,1367,3731,1592,3898,100,بمحمد
40271,226,0,0,20,1173,3731,1366,3898,99,وآله


In [3]:
afifi[afifi.letters.str.contains('\\d',regex=True)]

,page,stripe,column,line,left,top,right,bottom,confidence,letters


Normalize the text

In [4]:
afifi['letters'] = afifi['letters'].apply(abc.normalize)

What does Afifi's text consist of?

In [12]:
letters = set()
for word in afifi.letters.tolist():
    used = set(word)
    letters = letters | used
alphabeticalLetters = sorted(list(letters))
for a in alphabeticalLetters:
    print(a)

ء
آ
أ
ؤ
إ
ئ
ا
ب
ة
ت
ث
ج
ح
خ
د
ذ
ر
ز
س
ش
ص
ض
ط
ظ
ع
غ
ف
ق
ك
ل
م
ن
ه
و
ى
ي


In [15]:
# afifi = afifi[afifi.letters.str.len()!=0]
# afifi.to_csv('AfifiCleaned.csv',index=False)

14,288 = delete


In [120]:
print('Number of single letter entries: '+str(afifi[afifi.letters.str.len()==1].shape[0]))
for i in afifi[afifi.letters.str.len()==1].index:
    context = afifi.loc[i-2].letters + " " + afifi.loc[i-1].letters + " _" + afifi.loc[i].letters + "_ " + afifi.loc[i+1].letters + " " + afifi.loc[i+2].letters
    page = str(afifi.loc[i].page)
    line = str(afifi.loc[i].line)
    print("row " +str(i) + "    page "+ page + " line "+line)
    print(context + '\n')

Number of single letter entries: 320
row 14    page 47 line 2
من المقام _ا_ الأقدم وإن

row 288    page 48 line 15
يبلغها الاحصاء _س_ أن يرى

row 297    page 48 line 16
يرى عينه _ي_ كونجامع يحصرالأمر

row 416    page 49 line 8
ابتدأ منه _ا_ فاقتضى الأمر

row 489    page 49 line 14
حقيقة الحقائق _و_ في النشأة

row 530    page 50 line 1
هذا المذكور _م_ إنسانا وخليفة

row 568    page 50 line 5
الإنسان الحادث _ب_ الأزلي والنشء

row 745    page 51 line 1
الأسماء الإلمية _ه_ التي تخصها

row 1055    page 52 line 7
متميزة عنه _م_ نقول في

row 1211    page 53 line 1
لم تتفصل _ه_ ولم تتعدد

row 1270    page 53 line 6
أن المحدث _ب_ قد ثبت

row 1943    page 56 line 8
قوله تعالى _ا_ أيها الناس

row 1959    page 56 line 10
رجالا كثيرا _و_ نساء فقوله

row 2074    page 57 line 1
عليه وسلم _ٔ_ كمة إلمية

row 2502    page 59 line 15
وصنف يعلمون _ن_ استعدادهم ما

row 2842    page 61 line 1
الأحوال عليهإلى _ح_ ما لا

row 4053    page 65 line 19
علم شيث _ب_ عليه السلام

row 4193    page 66 line 11
وكل عطاء

In [10]:
afifi[afifi.letters.str.contains(abc.HAMZA_BELOW)]

,page,stripe,column,line,left,top,right,bottom,confidence,letters
0,47,0,0,1,1785,99,2375,441,56,بنزٕليلالع
494,49,0,0,15,2743,2853,2835,3044,88,ٕلى
674,50,0,0,13,2135,2332,2382,2519,100,الٕلهية
2246,58,0,0,10,1400,1839,1575,2011,98,ٕليها
5374,71,0,0,15,1766,2666,1939,2834,100,ٕلا
...,...,...,...,...,...,...,...,...,...,...
38128,217,0,0,19,2206,3473,2452,3646,97,الٕلمي
38164,218,0,0,4,2218,769,2325,937,91,ٕلى
38208,218,0,0,7,850,1314,1010,1479,100,ٕنما
39006,221,0,0,19,2391,3450,2670,3618,93,ٕولئكهم


In [11]:
afifi.letters = afifi.letters.str.replace(abc.HAMZA_BELOW,"")

In [14]:
afifi[afifi.letters.str.len()==0]

,page,stripe,column,line,left,top,right,bottom,confidence,letters
2074,57,0,0,1,1312,99,1363,198,98,
6304,76,0,0,3,967,598,996,785,100,
